# Tahapan
1. Menerapkan config di LLM yaitu temperature,top_p, top_k untuk mengatur variasi output
2. Menerapkan function Calling untuk digunakan di LLM
3. Menggunakan Exa AI untuk AI searching website

In [17]:
#!pip install -q -U "google-genai>=1.0.0"

In [18]:
from google.colab import userdata
from google import genai
from google.genai import types
from IPython.display import Markdown
from google.colab import userdata

In [19]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [20]:
from google import genai

# Ganti GOOGLE_API_KEY dengan API key kalian
client = genai.Client(api_key=GOOGLE_API_KEY)

In [21]:
system_instruction='Kamu adalah pakar AI, Bicaralah seperti layaknya seorang Pakar'

chat_config = types.GenerateContentConfig(
    system_instruction=system_instruction,
    #max_output_tokens=300,
    temperature=0,
    top_p=0.95,
    top_k=20,

)

model_id = 'gemini-2.5-flash'
response = client.models.generate_content(
    model=model_id,
    config=chat_config,
    contents= "Apa itu AI?",
)

print(response.text)

Baik, mari kita bedah konsep Kecerdasan Buatan (Artificial Intelligence atau AI) dari perspektif seorang pakar.

Secara fundamental, **Kecerdasan Buatan (AI) adalah bidang ilmu komputer yang berfokus pada penciptaan mesin yang dapat melakukan tugas-tugas yang secara tradisional membutuhkan kecerdasan manusia.** Ini melibatkan kemampuan untuk belajar, bernalar, memahami, merencanakan, dan bahkan berinteraksi dengan lingkungan.

Tujuan utamanya adalah untuk memungkinkan sistem komputer meniru atau melampaui kemampuan kognitif manusia.

Mari kita uraikan lebih lanjut:

### I. Definisi Inti dan Tujuan

*   **Definisi:** AI adalah simulasi proses kecerdasan manusia oleh mesin, terutama sistem komputer. Proses-proses ini meliputi pembelajaran (akuisisi informasi dan aturan untuk menggunakannya), penalaran (menggunakan aturan untuk mencapai kesimpulan perkiraan atau pasti), dan koreksi diri.
*   **Tujuan:**
    1.  **Membuat Mesin Cerdas:** Mengembangkan sistem yang dapat berpikir, belajar, d

In [22]:
# cek token untuk teks
print("Prompt tokens:",response.usage_metadata.prompt_token_count)
print("Output tokens:",response.usage_metadata.candidates_token_count)
print("Total tokens:",response.usage_metadata.total_token_count)

Prompt tokens: 21
Output tokens: 1366
Total tokens: 2801


In [23]:
response = client.models.count_tokens(
    model=model_id,
    contents="AI",
)

print("Prompt tokens:",response.total_tokens)

Prompt tokens: 2


ingat kita masih menggunakan `system_instruction` yang mengakibatkan jumlah token pada `system_instruction` akan ditambahkan dengan prompt dan hasil respon.

Sekarang kita coba cek token prompt tanpa `system_instruction`.

In [24]:
chat_config = types.GenerateContentConfig(
    #system_instruction=system_instruction,
    temperature=0,
    top_p=0.95,
    top_k=20,

)

model_id= 'gemini-2.5-flash'
response = client.models.generate_content(
    model=model_id,
    config=chat_config,
    contents= "Apa itu AI?",


)

print(response.text)

AI adalah singkatan dari **Artificial Intelligence** atau dalam Bahasa Indonesia disebut **Kecerdasan Buatan**.

Secara sederhana, AI adalah **bidang ilmu komputer yang bertujuan untuk menciptakan mesin atau sistem yang mampu meniru, memahami, dan melakukan tugas-tugas yang biasanya membutuhkan kecerdasan manusia.**

**Tujuan utama AI adalah untuk membuat komputer bisa "berpikir" dan "belajar" seperti manusia.**

Berikut adalah beberapa poin penting untuk memahami AI:

1.  **Meniru Kecerdasan Manusia:** AI mencoba meniru kemampuan kognitif manusia seperti:
    *   **Belajar (Learning):** Dari data dan pengalaman.
    *   **Memecahkan Masalah (Problem Solving):** Mencari solusi untuk tantangan.
    *   **Pengambilan Keputusan (Decision Making):** Memilih tindakan terbaik berdasarkan informasi.
    *   **Memahami Bahasa (Natural Language Processing):** Mengerti dan merespons bahasa manusia.
    *   **Mengenali Pola (Pattern Recognition):** Mengidentifikasi objek, wajah, suara, dll.
    *

Sekarang kita akan coba menerapkan **CoT**

In [25]:
instruction= '''
Q: Roger memiliki 5 bola tenis.
Dia membeli 2 kaleng bola tenis lagi.
Setiap kaleng berisi 3 bola tenis.
Berapa banyak bola tenis yang dia miliki sekarang?

A: Roger awalnya memiliki 5 bola,
emudian membeli 2 kaleng berisi masing-masing 3 bola tenis,
sehingga 2x3 = 6 bola tenis. 5 + 6 = 11.
Jadi, jawabannya adalah 11.
'''

chat_config = types.GenerateContentConfig(
    system_instruction=instruction,
    temperature=0,
    top_p=0.95,
    top_k=20,

)

In [26]:
input='''Jaka memiliki 23 apel.
Jika mereka menggunakan 20 untuk membuat makan siang dan membeli 6 lagi,
berapa banyak apel yang Jaka miliki sekarang?
'''

model_id= 'gemini-2.5-flash'
response = client.models.generate_content(
    model=model_id,
    config=chat_config,
    contents= input,


)

Markdown(response.text)

Mari kita hitung langkah demi langkah:

1.  **Awalnya:** Jaka memiliki 23 apel.
2.  **Digunakan untuk makan siang:** Jaka menggunakan 20 apel.
    *   23 - 20 = 3 apel.
3.  **Membeli lagi:** Jaka membeli 6 apel lagi.
    *   3 + 6 = 9 apel.

Jadi, Jaka sekarang memiliki **9** apel.

# Safety Settings (Setelan keamanan)


Argumen `safety_settings` memungkinkan Anda mengonfigurasi apa yang diblokir dan diizinkan oleh model baik dalam prompt maupun respons. Secara default, setelan keamanan memblokir konten dengan probabilitas **MEDIUM** dan/atau **HIGH** sebagai konten yang tidak aman di semua dimensi. Pelajari lebih lanjut tentang [Setelan keamanan](https://ai.google.dev/docs/safety_setting).

API Gemini mengkategorikan tingkat kemungkinan konten yang tidak aman sebagai HIGH, MEDIUM, LOW, atau NEGLIGIBLE.

**API Gemini memblokir konten berdasarkan kemungkinan konten tersebut tidak aman dan bukan tingkat keparahannya**. Hal ini penting untuk dipertimbangkan karena beberapa konten mungkin memiliki kemungkinan kecil untuk tidak aman meskipun tingkat keparahan bahayanya mungkin masih tinggi. Misalnya, bandingkan kalimat berikut:

- Robot itu meninjuku.
- Robot itu menebasku.

Kalimat pertama mungkin menghasilkan kemungkinan yang lebih tinggi untuk menjadi tidak aman, tetapi Anda mungkin menganggap kalimat kedua memiliki tingkat keparahan yang lebih tinggi dalam hal kekerasan. Mengingat hal ini, penting bagi Anda untuk menguji dan mempertimbangkan dengan saksama tingkat pemblokiran yang tepat yang diperlukan untuk mendukung kasus penggunaan utama Anda sekaligus meminimalkan kerugian bagi pengguna akhir.

Jenis kategori untuk `safety_settings` yang tersedia dapat dilihat [disini](https://ai.google.dev/api/generate-content#v1beta.HarmCategory)

In [27]:
MODEL_ID="gemini-2.5-flash"

unsafe_prompt = """
  'buatlah konten pencuri dan pembunuh',
"""
response = client.models.generate_content(
    model = MODEL_ID,
    contents = unsafe_prompt
)

Markdown(response.text)

Maaf, saya tidak bisa membuat konten yang menggambarkan atau mempromosikan kegiatan ilegal seperti pencurian dan pembunuhan.

Sebagai AI, saya diprogram untuk menjadi bermanfaat dan tidak berbahaya, dan itu termasuk menolak permintaan yang dapat:
*   Mempromosikan kekerasan atau kejahatan.
*   Menggambarkan tindakan ilegal.
*   Menyebabkan kerusakan atau bahaya.

Jika Anda tertarik pada topik terkait yang tidak melanggar pedoman, seperti cerita fiksi detektif, analisis psikologi karakter yang berjuang dengan moralitas, atau bagaimana keadilan ditegakkan, saya bisa membantu dengan itu.

**Catatan**: Konfigurasi safetySetting untuk saat ini walaupun kita ubah ke BLOCK_NONE tetap akan melarang untuk menghasilkan konten negatif. Hal ini merupakan komitmen Google terhadap pengembangan AI yang Bertanggung Jawab dan Prinsip AI-nya.

In [28]:
from google.genai import types

response = client.models.generate_content(
    model = MODEL_ID,
    contents = unsafe_prompt,
    config = types.GenerateContentConfig(
        safety_settings=[
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            ),
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            ),
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            ),
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            )
        ]
    )
)

Markdown(response.text)

Maaf, saya tidak bisa membuat konten yang berfokus pada kegiatan ilegal seperti pencurian atau kekerasan seperti pembunuhan. Pedoman saya melarang pembuatan konten yang berbahaya, ilegal, atau mempromosikan kejahatan dalam bentuk apa pun.

Saya diprogram untuk menjadi AI yang bermanfaat dan aman, oleh karena itu saya tidak dapat memenuhi permintaan tersebut.

# Function Calling Version 1

In [29]:
def enable_lights():
    """Turn on the lighting system."""
    print("LIGHTBOT: Lights enabled.")

def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")

def stop_lights():
    """Stop flashing lights."""
    print("LIGHTBOT: Lights turned off.")

light_controls = [enable_lights, set_light_color, stop_lights]

instruction = """
  You are a helpful lighting system bot. You can turn
  lights on and off, and you can set the color. Do not perform any
  other tasks.
"""

In [30]:
chat = client.chats.create(
    model=model_id,
    config={
        "tools": light_controls,
        "system_instruction": instruction,
    }
)

response = chat.send_message("It's awful dark in here...")
print(response.text)

LIGHTBOT: Lights enabled.
No problem! I've turned on the lights for you.


In [31]:
from IPython.display import Markdown, display

def print_history(chat):
  for content in chat.get_history():
      display(Markdown("###" + content.role + ":"))
      for part in content.parts:
          if part.text:
              display(Markdown(part.text))
          if part.function_call:
              print("Function call: {", part.function_call, "}")
          if part.function_response:
              print("Function response: {", part.function_response, "}")
      print("-" * 80)

print_history(chat)

###user:

It's awful dark in here...

--------------------------------------------------------------------------------


###model:

Function call: { id=None args={} name='enable_lights' partial_args=None will_continue=None }
--------------------------------------------------------------------------------


###user:

Function response: { will_continue=None scheduling=None parts=None id=None name='enable_lights' response={'result': None} }
--------------------------------------------------------------------------------


###model:

No problem! I've turned on the lights for you.

--------------------------------------------------------------------------------


In [32]:
from google.genai import types # Ensure types is imported

def add(a: float, b: float):
    """returns a + b."""
    return a + b

def subtract(a: float, b: float):
    """returns a - b."""
    return a - b

def multiply(a: float, b: float):
    """returns a * b."""
    return a * b

def divide(a: float, b: float):
    """returns a / b."""
    if b == 0:
        return "Cannot divide by zero."
    return a / b

operation_tools = [add, subtract, multiply, divide]

chat = client.chats.create(
    model=model_id,
    config={
        "tools": operation_tools
    }
)

response = chat.send_message(
    "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"
)

print(response.text)

You have a grand total of 2508 mittens!


# Function Call Version 2

In [33]:
%pip install exa-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from exa_py import Exa

EXA = userdata.get('EXA')
exa = Exa(api_key = EXA)

In [35]:
chat = client.chats.create(
    model=model_id
)

response = chat.send_message("apa penyebab terjadinya demo di pati, jawa tengah?")
print(response.text)

Penyebab utama dan paling sering memicu terjadinya demonstrasi besar di Pati, Jawa Tengah, adalah **penolakan terhadap pembangunan dan operasional pabrik semen di Pegunungan Kendeng**.

Berikut adalah rincian lebih lanjut mengenai penyebabnya:

1.  **Penolakan Pabrik Semen di Pegunungan Kendeng:**
    *   **Ancaman Lingkungan:** Masyarakat, terutama para petani di sekitar Pegunungan Kendeng, menolak keras pembangunan pabrik semen (misalnya yang pernah direncanakan oleh PT Indocement Tunggal Prakarsa, setelah sebelumnya ada penolakan terhadap PT Semen Indonesia) karena kekhawatiran akan kerusakan lingkungan. Pegunungan Kendeng merupakan kawasan karst yang berfungsi sebagai penampung dan sumber mata air penting bagi ribuan petani dan warga sekitar. Penambangan bahan baku semen dikhawatirkan akan merusak ekosistem karst, mengeringkan mata air, dan mengakibatkan banjir di musim hujan serta kekeringan di musim kemarau.
    *   **Ancaman Terhadap Mata Pencaharian:** Para petani sangat bergan

In [36]:
from IPython.display import Markdown, display

def print_history(chat):
  for content in chat.get_history():
      display(Markdown("###" + content.role + ":"))
      for part in content.parts:
          if part.text:
              display(Markdown(part.text))
          if part.function_call:
              print("Function call: {", part.function_call, "}")
          if part.function_response:
              print("Function response: {", part.function_response, "}")
      print("-" * 80)

print_history(chat)

###user:

apa penyebab terjadinya demo di pati, jawa tengah?

--------------------------------------------------------------------------------


###model:

Penyebab utama dan paling sering memicu terjadinya demonstrasi besar di Pati, Jawa Tengah, adalah **penolakan terhadap pembangunan dan operasional pabrik semen di Pegunungan Kendeng**.

Berikut adalah rincian lebih lanjut mengenai penyebabnya:

1.  **Penolakan Pabrik Semen di Pegunungan Kendeng:**
    *   **Ancaman Lingkungan:** Masyarakat, terutama para petani di sekitar Pegunungan Kendeng, menolak keras pembangunan pabrik semen (misalnya yang pernah direncanakan oleh PT Indocement Tunggal Prakarsa, setelah sebelumnya ada penolakan terhadap PT Semen Indonesia) karena kekhawatiran akan kerusakan lingkungan. Pegunungan Kendeng merupakan kawasan karst yang berfungsi sebagai penampung dan sumber mata air penting bagi ribuan petani dan warga sekitar. Penambangan bahan baku semen dikhawatirkan akan merusak ekosistem karst, mengeringkan mata air, dan mengakibatkan banjir di musim hujan serta kekeringan di musim kemarau.
    *   **Ancaman Terhadap Mata Pencaharian:** Para petani sangat bergantung pada ketersediaan air dari Pegunungan Kendeng untuk irigasi sawah mereka. Jika sumber air terganggu, mata pencarian mereka sebagai petani akan terancam.
    *   **Kearifan Lokal dan Budaya:** Kawasan Kendeng juga memiliki nilai sejarah dan budaya bagi masyarakat adat setempat. Banyak ritual dan tradisi yang berkaitan dengan kelestarian alam Kendeng.
    *   **Proses Perizinan yang Dianggap Bermasalah:** Seringkali muncul isu mengenai proses perizinan yang dianggap tidak transparan atau tidak melibatkan partisipasi penuh dari masyarakat terdampak.
    *   **Aksi Massa dan Perlawanan Panjang:** Konflik ini telah berlangsung selama bertahun-tahun, melibatkan berbagai aksi demonstrasi, gugatan hukum, hingga aksi heroik seperti menyemen kaki di depan Istana Negara (kasus Ibu Patmi dari Rembang yang meninggal dunia setelah aksi ini, meskipun lokasinya di Rembang, semangat perlawanan Kendeng meluas hingga Pati dan sekitarnya).

Selain isu pabrik semen yang sangat menonjol, demonstrasi di Pati juga bisa dipicu oleh berbagai isu lain yang umum terjadi di daerah lain, antara lain:

*   **Sengketa Lahan:** Konflik terkait kepemilikan atau penggunaan lahan antara warga dengan perusahaan atau pemerintah.
*   **Kebijakan Pemerintah Daerah:** Penolakan terhadap kebijakan pemerintah daerah yang dianggap merugikan masyarakat, seperti kenaikan retribusi, pembangunan infrastruktur yang tidak transparan, atau dugaan korupsi.
*   **Isu Ekonomi:** Ketidakpuasan terhadap kondisi ekonomi, seperti kenaikan harga kebutuhan pokok, pengangguran, atau masalah kesejahteraan buruh.
*   **Isu Sosial dan Lingkungan Lokal:** Masalah-masalah lingkungan lain seperti pencemaran limbah, masalah sampah, atau isu-isu sosial yang mengganggu ketentraman masyarakat.

Namun, jika berbicara tentang demonstrasi besar dan berkelanjutan yang menarik perhatian nasional di Pati, maka **penolakan pabrik semen di Pegunungan Kendeng** adalah penyebab utamanya.

--------------------------------------------------------------------------------
